<a href="https://colab.research.google.com/github/Mraghuvaran/t-SNE-Visualization/blob/master/10_09_2020_Iteration_model_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://sagemaker-studio-944864437887-vpkn7w39q58.s3.amazonaws.com/Original_data.csv

--2020-09-11 08:43:05--  https://sagemaker-studio-944864437887-vpkn7w39q58.s3.amazonaws.com/Original_data.csv
Resolving sagemaker-studio-944864437887-vpkn7w39q58.s3.amazonaws.com (sagemaker-studio-944864437887-vpkn7w39q58.s3.amazonaws.com)... 52.216.18.72
Connecting to sagemaker-studio-944864437887-vpkn7w39q58.s3.amazonaws.com (sagemaker-studio-944864437887-vpkn7w39q58.s3.amazonaws.com)|52.216.18.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96790966 (92M) [text/csv]
Saving to: ‘Original_data.csv.3’

Original_data.csv.3 100%[===================>]  92.31M  37.3MB/s    in 2.5s    

2020-09-11 08:43:08 (37.3 MB/s) - ‘Original_data.csv.3’ saved [96790966/96790966]



In [2]:
# Importing libraries 
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

from mlxtend.evaluate import feature_importance_permutation

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sn

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
#reading the data 
data = pd.read_csv("/content/Original_data.csv",low_memory=False)
data.head()

,ADDRESS_X,LATITUDE_X,LONGITUDE_X,AGE,COMMUNITY_COUNCIL_NEIGHBORHOOD,CPD_NEIGHBORHOOD,CRASHDATE,CRASHLOCATION,CRASHSEVERITY,CRASHSEVERITYID,DATECRASHREPORTED,DAYOFWEEK,GENDER,INJURIES,INSTANCEID,LIGHTCONDITIONSPRIMARY,LOCALREPORTNO,MANNEROFCRASH,ROADCONDITIONSPRIMARY,ROADCONTOUR,ROADSURFACE,SNA_NEIGHBORHOOD,TYPEOFPERSON,WEATHER,ZIP,UNITTYPE
0,NaN,NaN,NaN,UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9C12DABA-4EE6-4283-9443-CE45BE543EDC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,UNKNOWN,NaN,NaN,10/05/2019 09:15:00 AM,NaN,5 - PROPERTY DAMAGE ONLY,201905.0,NaN,SAT,NaN,NaN,E6FD3246-699E-46DB-8CBE-A908C2E7D7FE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13F29735-61AC-4116-B41E-A58C66D3F991,NaN,135001314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13F29735-61AC-4116-B41E-A58C66D3F991,NaN,135001314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38160487-D58E-4D75-BBB9-B3D3C82F3D20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#checking the column names 
data.columns

Index(['ADDRESS_X', 'LATITUDE_X', 'LONGITUDE_X', 'AGE',
       'COMMUNITY_COUNCIL_NEIGHBORHOOD', 'CPD_NEIGHBORHOOD', 'CRASHDATE',
       'CRASHLOCATION', 'CRASHSEVERITY', 'CRASHSEVERITYID',
       'DATECRASHREPORTED', 'DAYOFWEEK', 'GENDER', 'INJURIES', 'INSTANCEID',
       'LIGHTCONDITIONSPRIMARY', 'LOCALREPORTNO', 'MANNEROFCRASH',
       'ROADCONDITIONSPRIMARY', 'ROADCONTOUR', 'ROADSURFACE',
       'SNA_NEIGHBORHOOD', 'TYPEOFPERSON', 'WEATHER', 'ZIP', 'UNITTYPE'],
      dtype='object')

In [5]:
#finding the number of unique values in each column 
n = data.nunique(axis=0) 
  
print("No.of.unique values in each column :\n", 
      n)

No.of.unique values in each column :
 ADDRESS_X                          12268
LATITUDE_X                         92838
LONGITUDE_X                       122939
AGE                                    9
COMMUNITY_COUNCIL_NEIGHBORHOOD        71
CPD_NEIGHBORHOOD                      53
CRASHDATE                         117149
CRASHLOCATION                         13
CRASHSEVERITY                          8
CRASHSEVERITYID                        8
DATECRASHREPORTED                 119797
DAYOFWEEK                              7
GENDER                                 5
INJURIES                              10
INSTANCEID                        123566
LIGHTCONDITIONSPRIMARY                14
LOCALREPORTNO                     123504
MANNEROFCRASH                          9
ROADCONDITIONSPRIMARY                 11
ROADCONTOUR                            5
ROADSURFACE                            8
SNA_NEIGHBORHOOD                      50
TYPEOFPERSON                           3
WEATHER            

In [6]:
#converting the column names to lower case 
data.columns= data.columns.str.lower()
data.head()

,address_x,latitude_x,longitude_x,age,community_council_neighborhood,cpd_neighborhood,crashdate,crashlocation,crashseverity,crashseverityid,datecrashreported,dayofweek,gender,injuries,instanceid,lightconditionsprimary,localreportno,mannerofcrash,roadconditionsprimary,roadcontour,roadsurface,sna_neighborhood,typeofperson,weather,zip,unittype
0,NaN,NaN,NaN,UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9C12DABA-4EE6-4283-9443-CE45BE543EDC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,UNKNOWN,NaN,NaN,10/05/2019 09:15:00 AM,NaN,5 - PROPERTY DAMAGE ONLY,201905.0,NaN,SAT,NaN,NaN,E6FD3246-699E-46DB-8CBE-A908C2E7D7FE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13F29735-61AC-4116-B41E-A58C66D3F991,NaN,135001314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13F29735-61AC-4116-B41E-A58C66D3F991,NaN,135001314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38160487-D58E-4D75-BBB9-B3D3C82F3D20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#dropping un-necessary columns 
data.drop(["instanceid", "community_council_neighborhood","cpd_neighborhood","crashdate","localreportno",
           "latitude_x","longitude_x","crashseverityid","datecrashreported","address_x"], axis=1, inplace=True)

In [8]:
#dropping all the rows with missing values 
data.dropna(axis = 0, how = 'any', inplace=True)

In [9]:
# filling all na with UNKNOWN string, so we can encode later
# data.fillna('NA', inplace=True)

In [10]:
data.head()

,age,crashlocation,crashseverity,dayofweek,gender,injuries,lightconditionsprimary,mannerofcrash,roadconditionsprimary,roadcontour,roadsurface,sna_neighborhood,typeofperson,weather,zip,unittype
40,18-25,07 - ON RAMP,2 - INJURY,SAT,M - MALE,2 - POSSIBLE,4 - DARK - LIGHTED ROADWAY,2 - REAR-END,01 - DRY,1 - STRAIGHT LEVEL,1 - CONCRETE,AVONDALE,D - DRIVER,1 - CLEAR,45209,02 - COMPACT
41,UNDER 18,02 - FOUR-WAY INTERSECTION,2 - INJURY,THU,F - FEMALE,4 - INCAPACITATING,1 - DAYLIGHT,6 - ANGLE,01 - DRY,1 - STRAIGHT LEVEL,"2 - BLACKTOP, BITUMINOUS, ASPHALT",WESTWOOD,O - OCCUPANT,1 - CLEAR,45211,03 - MID SIZE
47,31-40,03 - T-INTERSECTION,3 - PROPERTY DAMAGE ONLY (PDO),TUE,M - MALE,1 - NO INJURY / NONE REPORTED,4 - DARK - LIGHTED ROADWAY,"7 - SIDESWIPE, SAME DIRECTION",02 - WET,3 - CURVE LEVEL,"2 - BLACKTOP, BITUMINOUS, ASPHALT",COLLEGE HILL,D - DRIVER,2 - CLOUDY,45239,03 - MID SIZE
49,51-60,01 - NOT AN INTERSECTION,3 - PROPERTY DAMAGE ONLY (PDO),WED,M - MALE,1 - NO INJURY / NONE REPORTED,1 - DAYLIGHT,"7 - SIDESWIPE, SAME DIRECTION",01 - DRY,1 - STRAIGHT LEVEL,"2 - BLACKTOP, BITUMINOUS, ASPHALT",CORRYVILLE,D - DRIVER,1 - CLEAR,45219,06 - SPORT UTILITY VEHICLE
50,61-70,01 - NOT AN INTERSECTION,2 - INJURY,SAT,F - FEMALE,3 - NON-INCAPACITATING,1 - DAYLIGHT,1 - NOT COLLISION BETWEEN TWO MOTOR VEHICLES I...,01 - DRY,1 - STRAIGHT LEVEL,"2 - BLACKTOP, BITUMINOUS, ASPHALT",WEST END,P - PEDESTRIAN,1 - CLEAR,45203,26 - PEDESTRIAN/SKATER


In [11]:
# Useful columns for further experiment
data.columns

Index(['age', 'crashlocation', 'crashseverity', 'dayofweek', 'gender',
       'injuries', 'lightconditionsprimary', 'mannerofcrash',
       'roadconditionsprimary', 'roadcontour', 'roadsurface',
       'sna_neighborhood', 'typeofperson', 'weather', 'zip', 'unittype'],
      dtype='object')

In [12]:
#onehot encodig the manner of crash column 
ohe = OneHotEncoder(handle_unknown='ignore')
crash_type = data['mannerofcrash']
crash_df = pd.DataFrame(crash_type, columns=['mannerofcrash'])
crash_df = pd.DataFrame(ohe.fit_transform(crash_df[['mannerofcrash']]).toarray())

In [13]:
#filtering the data by mannerofcrash
crash_col = data['mannerofcrash'].unique()
crash_df.columns = crash_col

In [14]:
crash_df.columns

Index(['2 - REAR-END', '6 - ANGLE', '7 - SIDESWIPE, SAME DIRECTION',
       '1 - NOT COLLISION BETWEEN TWO MOTOR VEHICLES IN TRANSPORT',
       '4 - REAR-TO-REAR', '8 - SIDESWIPE, OPPOSITE DIRECTION', '3 - HEAD-ON',
       '5 - BACKING', '9 - UNKNOWN'],
      dtype='object')

In [15]:
# We will also create a new dataframe for them
text_col = ['age', 'sna_neighborhood', 'crashseverity', 'dayofweek',
       'crashlocation', 'gender', 'injuries', 'lightconditionsprimary',
       'roadconditionsprimary', 'roadcontour', 'roadsurface',
       'typeofperson', 'weather', 'unittype', 'zip']
en_df = pd.DataFrame()
for col in text_col:
    encoder = LabelEncoder()
#     data[cat] = le.fit_transform(df[cat].astype(str))

    en_df[col] = encoder.fit_transform(data[col].astype(str))
    
en_df.head()

,age,sna_neighborhood,crashseverity,dayofweek,crashlocation,gender,injuries,lightconditionsprimary,roadconditionsprimary,roadcontour,roadsurface,typeofperson,weather,unittype,zip
0,0,0,1,2,6,2,1,5,0,0,0,0,0,2,18
1,7,48,1,4,1,0,5,0,0,0,1,1,0,4,20
2,2,6,4,5,2,2,0,5,1,2,1,0,1,4,47
3,4,8,4,6,0,2,0,0,0,0,1,0,0,8,28
4,5,46,1,2,0,0,3,0,0,0,1,2,0,30,12


In [16]:

# Merge the FIRST_CRASH_TYPE and the LabelEncoded dataframe together
df2 = pd.merge(crash_df, en_df, left_index=True, right_index=True)

In [17]:
df2.head()

,2 - REAR-END,6 - ANGLE,"7 - SIDESWIPE, SAME DIRECTION",1 - NOT COLLISION BETWEEN TWO MOTOR VEHICLES IN TRANSPORT,4 - REAR-TO-REAR,"8 - SIDESWIPE, OPPOSITE DIRECTION",3 - HEAD-ON,5 - BACKING,9 - UNKNOWN,age,sna_neighborhood,crashseverity,dayofweek,crashlocation,gender,injuries,lightconditionsprimary,roadconditionsprimary,roadcontour,roadsurface,typeofperson,weather,unittype,zip
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,2,6,2,1,5,0,0,0,0,0,2,18
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,7,48,1,4,1,0,5,0,0,0,1,1,0,4,20
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2,6,4,5,2,2,0,5,1,2,1,0,1,4,47
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4,8,4,6,0,2,0,0,0,0,1,0,0,8,28
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,46,1,2,0,0,3,0,0,0,1,2,0,30,12


In [18]:
df2.columns

Index(['2 - REAR-END', '6 - ANGLE', '7 - SIDESWIPE, SAME DIRECTION',
       '1 - NOT COLLISION BETWEEN TWO MOTOR VEHICLES IN TRANSPORT',
       '4 - REAR-TO-REAR', '8 - SIDESWIPE, OPPOSITE DIRECTION', '3 - HEAD-ON',
       '5 - BACKING', '9 - UNKNOWN', 'age', 'sna_neighborhood',
       'crashseverity', 'dayofweek', 'crashlocation', 'gender', 'injuries',
       'lightconditionsprimary', 'roadconditionsprimary', 'roadcontour',
       'roadsurface', 'typeofperson', 'weather', 'unittype', 'zip'],
      dtype='object')

Splitting the data into X & Y 

In [19]:
y = df2['crashseverity']
X = df2.loc[:, df2.columns != 'crashseverity']

In [20]:
# Train and Test on a Vanilla model (test size to 20%)
dt_model = DecisionTreeClassifier(random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

In [21]:
dt_model.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1, splitter='best')

In [22]:
y_train_pred = dt_model.predict(X_train)
y_test_pred = dt_model.predict(X_test)

In [23]:
print(f'Accuracy Score Train: {accuracy_score(y_train, y_train_pred)}')
print(f'Accuracy Score Test: {accuracy_score(y_test, y_test_pred)}')

Accuracy Score Train: 0.9941027049292874
Accuracy Score Test: 0.8193388940376892


In [24]:

rf = RandomForestClassifier(class_weight='balanced', random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)
rf.fit(X_train, y_train)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [25]:
y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)

print(f'Accuracy Score Train: {accuracy_score(y_train, y_train_pred)}')
print(f'Accuracy Score Test: {accuracy_score(y_test, y_test_pred)}')

Accuracy Score Train: 0.9939653988740903
Accuracy Score Test: 0.882483781278962


In [26]:
## Random forest classifier using grid search cv 

In [27]:
# Here we set a some parameters that the GridSearch will run through.
grid_p = {"n_estimators": [100, 150],
          "criterion": ["gini"],
          "max_depth": [12, 14, 17],
          "min_samples_split": [4,5],
          "min_samples_leaf": [4,5,6,]
         }

grid_search = GridSearchCV(rf, grid_p, n_jobs=-1, cv=10, scoring='accuracy')
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


GridSearchCV(cv=10, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight='balanced',
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=1,
                            

In [28]:
grid_search.best_score_

0.8893999725387889

In [29]:
grid_search.best_params_

{'criterion': 'gini',
 'max_depth': 17,
 'min_samples_leaf': 4,
 'min_samples_split': 4,
 'n_estimators': 150}

In [30]:
y_train_pred = grid_search.predict(X_train)
y_test_pred = grid_search.predict(X_test)

print(f'Accuracy Score Train: {accuracy_score(y_train, y_train_pred)}')
print(f'Accuracy Score Test: {accuracy_score(y_test, y_test_pred)}')

Accuracy Score Train: 0.9051077852533297
Accuracy Score Test: 0.8852023478529503
